# Model Evaluation with K-Fold & Grid Search

- Breast Cancer Dataset (in-built dataset)


### Step 1: Import necessary libraries


In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
import warnings
warnings.filterwarnings("ignore")

### Step 2: Load the Breast Cancer dataset


In [2]:
data = load_breast_cancer()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = pd.Series(data.target)

### Step 3: Split the dataset into training and testing sets


In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


### Step 4: Create a Logistic Regression classifier


In [4]:
clf = LogisticRegression(random_state=42)

### Step 5: Perform K-Fold Cross-Validation (K=5) to evaluate the model


In [5]:
cv_scores = cross_val_score(clf, X_train, y_train, cv=5, scoring='accuracy')

### Step 6: Fit the model on the entire training set


In [6]:
clf.fit(X_train, y_train)


LogisticRegression(random_state=42)

### Step 7: Make predictions on the test set


In [7]:
y_pred = clf.predict(X_test)


### Step 8: Evaluate the model using classification metrics


In [8]:
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

### Step 9: Print the K-Fold Cross-Validation results


In [9]:
print("K-Fold Cross-Validation Results:")
for i, score in enumerate(cv_scores, 1):
    print(f"Fold {i}: {score:.2f}")

K-Fold Cross-Validation Results:
Fold 1: 0.98
Fold 2: 0.90
Fold 3: 0.97
Fold 4: 0.96
Fold 5: 0.90


### Step 10: Print the evaluation metrics and classification report


In [10]:
print("\nTest Set Evaluation Metrics:")
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")
print("\nConfusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(classification_report(y_test, y_pred))


Test Set Evaluation Metrics:
Accuracy: 0.96
Precision: 0.95
Recall: 0.99
F1 Score: 0.97

Confusion Matrix:
[[39  4]
 [ 1 70]]

Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.91      0.94        43
           1       0.95      0.99      0.97        71

    accuracy                           0.96       114
   macro avg       0.96      0.95      0.95       114
weighted avg       0.96      0.96      0.96       114



### Step 11: Perform Grid Search for hyperparameter tuning


In [90]:
# C :- Regularization strength
# penalty :- Type of regularization
# solver :- Solvers for optimization
# dual :- Dual formulation (only with 'liblinear' solver)
# fit_intercept :- Whether to fit the intercept
# tol :- # Tolerance for stopping criteria
# max_iter :- # Maximum number of iterations
# class_weight :- Adjust weights based on class distribution


# Accuracy Model: 0.98
# param_grid = {
#     'C': [0.001, 0.01, 0.1, 1, 10, 100],      
#     'penalty': ['l1', 'l2', 'elasticnet', 'none'],  
#     'solver': ['liblinear', 'saga', 'lbfgs', 'newton-cg'],
#     'dual': [False, True], 
#     'fit_intercept': [True, False],      
# }

# ---------> NEW  <---------

param_grid = {
    # 'C': [0.001, 0.01, 0.1, 1],              
    'penalty': ['l1', 'l2', 'none'],    
    # 'solver': ['liblinear', 'newton-cg', 'lbfgs', 'sag', 'saga'],
    'dual': [False, True],                           
    # 'tol': [1e-4, 1e-3, 1e-1],                  
    'fit_intercept': [True, False],                    
    'class_weight': [None, 'balanced'],               
    'max_iter': [100, 200, 300, 500],                  
    'multi_class': ['auto', 'ovr', 'multinomial'],    
    'l1_ratio': [None, 0.1, 0.5, 0.7, 0.9],          
}

grid_search = GridSearchCV(estimator=clf, param_grid=param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=LogisticRegression(random_state=42),
             param_grid={'class_weight': [None, 'balanced'],
                         'dual': [False, True], 'fit_intercept': [True, False],
                         'l1_ratio': [None, 0.1, 0.5, 0.7, 0.9],
                         'max_iter': [100, 200, 300, 500],
                         'multi_class': ['auto', 'ovr', 'multinomial'],
                         'penalty': ['l1', 'l2', 'none']},
             scoring='accuracy')

### Step 12: Get the best hyperparameters


In [91]:
best_params = grid_search.best_params_
best_params

{'class_weight': 'balanced',
 'dual': False,
 'fit_intercept': True,
 'l1_ratio': None,
 'max_iter': 500,
 'multi_class': 'auto',
 'penalty': 'l2'}

### Step 13: Train a Logistic Regression classifier with the best hyperparameters


In [92]:
best_clf = LogisticRegression(random_state=42, **best_params)
best_clf.fit(X_train, y_train)

LogisticRegression(class_weight='balanced', max_iter=500, multi_class='auto',
                   random_state=42)

### Step 14: Make predictions with the tuned model


In [93]:
y_pred_tuned = best_clf.predict(X_test)


### Step 15: Evaluate the tuned model


In [94]:
accuracy_tuned = accuracy_score(y_test, y_pred_tuned)
print("\nGrid Search Results:")
print(f"Best Hyperparameters: {best_params}")
print(f"Test Set Accuracy with Tuned Model: {accuracy_tuned:.2f}")


Grid Search Results:
Best Hyperparameters: {'class_weight': 'balanced', 'dual': False, 'fit_intercept': True, 'l1_ratio': None, 'max_iter': 500, 'multi_class': 'auto', 'penalty': 'l2'}
Test Set Accuracy with Tuned Model: 0.96


### Compare result of k-fold & Grid Search


In [97]:
# Perform K-Fold Cross-Validation and store the accuracy scores
cv_scores = cross_val_score(clf, X_train, y_train, cv=5, scoring='accuracy')

# Calculate the accuracy of the tuned model
accuracy_tuned = accuracy_score(y_test, y_pred_tuned)

# Print and compare the results side by side
print("K-Fold Cross-Validation Results:")
print(f"Mean Accuracy: {np.mean(cv_scores):.2f}")
print(f"Test Set Accuracy: {accuracy:.2f}")

print("\nGrid Search Results:")
print(f"Best Hyperparameters: {best_params}")
print(f"Test Set Accuracy with Tuned Model: {accuracy_tuned:}")

K-Fold Cross-Validation Results:
Mean Accuracy: 0.94
Test Set Accuracy: 0.96

Grid Search Results:
Best Hyperparameters: {'class_weight': 'balanced', 'dual': False, 'fit_intercept': True, 'l1_ratio': None, 'max_iter': 500, 'multi_class': 'auto', 'penalty': 'l2'}
Test Set Accuracy with Tuned Model: 0.9649122807017544


In [98]:
# 0.956140350877193
#{
    # 'class_weight': 'balanced', 
    # 'dual': False, 
    # 'fit_intercept': True,
    # 'l1_ratio': None, 
    # 'max_iter': 500, 
    # 'multi_class': 'auto', 
    # 'penalty': 'l2'}
# 0.9649122807017544